In [2]:
import bokeh
import shutil
import sys
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from collections import defaultdict, namedtuple
from pathlib import Path
from queue import Queue
from time import sleep
from dateutil import parser
import tempfile
from watchdog.observers import Observer, ObserverType
from bokeh.plotting import figure
from bokeh.io import show, push_notebook
from bokeh.layouts import layout



from ogstools.logparser.log_file_handler import (
    LogFileHandler,
    normalize_regex,
    parse_line,
    select_regex,
)
from ogstools.logparser.regexes import (
    Context,
    StepStatus,
    Termination,
    TimeStepEnd,
    TimeStepStart,
    AssemblyTime,
    LinearSolverTime,
    IterationEnd,
    IterationStart,
    TimeStepConvergenceCriterion,
    ComponentConvergenceCriterion
)   
print("Bokeh Version : {}".format(bokeh.__version__))

Bokeh Version : 3.7.3


In [3]:
import requests
import time
from datetime import datetime
from bokeh.io import output_notebook
from bokeh.io import push_notebook

output_notebook()

Loading BokehJS ...

In [4]:
from bokeh.models import ColumnDataSource

data_source = ColumnDataSource(data = {"time_step": [], "step_size": [], "assembly_time": [], "linear_solver_time": [], "step_start_time": [], "iteration_number": []})
data_source_iter = ColumnDataSource(data = {"iteration_number": [], "vspan": [], "line_width": [], "dx_x": [], "dx_x_0": [], "dx_x_1": [],"dx_x_2": [], "dx_x_3": [], "dx_x_4": [],"dx_x_5": []})

In [5]:
chunk_size=4096
delay = 0.0

original_file = Path("out_test_transition-sand.log")
print(str(original_file.parent))
#temp_dir = Path(
#            tempfile.mkdtemp(
#                f"test_v2_coupled_with_producer_{chunk_size}_{delay}"
#            )
#        )

#temp_dir.mkdir(parents=True, exist_ok=True)

#new_file = temp_dir / "ht.log"
records: Queue = Queue()
observer: ObserverType = Observer()
status: Context = Context()
#shutil.rmtree(new_file, ignore_errors=True)
handler = LogFileHandler(
    original_file,
    queue=records,
    status=status,
    stop_callback=lambda: (print("Stop Observer"), observer.stop()),
)

observer.schedule(handler, path=str(original_file.parent), recursive=False)

print("Starting observer...")

observer.start()

.
Starting observer...


In [13]:
fig_1 = figure(  width=500, height=450,
             tooltips=[("step_size", "@step_size"),],
             title="Step Size", y_axis_type="log"
            )

fig_1.line(x="time_step", y="step_size", line_color="tomato", line_width=3.0, source=data_source)

fig_1.xaxis.axis_label = "time_step (s)"
fig_1.yaxis.axis_label = "step size (s)"

fig_2 = figure(  width=500, height=450,
             tooltips=[("assembly_time", "@assembly_time"),],
             title="Assembly Time"
            )

fig_2.line(x="time_step", y="assembly_time", line_color="tomato", line_width=3.0, source=data_source)
fig_2.line(x="time_step", y="linear_solver_time", line_color="blue", line_width=3.0, source=data_source)

fig_2.xaxis.axis_label = "time_step (s)"
fig_2.yaxis.axis_label = "assembly time (s)"

fig_3= figure(  width=500, height=450,
             tooltips=[("step_start_time", "@step_start_time"),],
             title="Time Step Start Time", y_axis_type="log"
            )

fig_3.line(x="time_step", y="step_start_time", line_color="tomato", line_width=3.0, source=data_source)

fig_3.xaxis.axis_label = "time_step (s)"
fig_3.yaxis.axis_label = "time (s)"

fig_4= figure(  width=500, height=450,
             tooltips=[("iteration_number", "@iteration_number"),],
             title="Iteration Number"
            )

fig_4.line(x="time_step", y="iteration_number", line_color="tomato", line_width=3.0, source=data_source)

fig_4.xaxis.axis_label = "time_step (s)"
fig_4.yaxis.axis_label = "number of iterations"

fig_5= figure(  width=500, height=450,
             tooltips=[("dx_x", "dx_x"),],
             title="Relative Convergence", y_axis_type="log"
            )

fig_5.line(x="iteration_number", y="dx_x", line_color="tomato", line_width=3.0, source=data_source_iter)
fig_5.vspan(x="vspan", line_width="line_width", line_color="blue", source=data_source_iter)

fig_5.xaxis.axis_label = "number of iterations"
fig_5.yaxis.axis_label = "relative convergence"

fig_6= figure(  width=500, height=450,
             tooltips=[("dx_x_0", "@dx_x_0"),],
             title="Relative Convergence", y_axis_type="log"
            )

fig_6.line(x="iteration_number", y="dx_x_0", line_color="tomato", line_width=3.0, source=data_source_iter)
fig_6.vspan(x="vspan", line_width="line_width", line_color="blue", source=data_source_iter)


fig_6.xaxis.axis_label = "number of iterations"
fig_6.yaxis.axis_label = "relative convergence"


fig = layout([[fig_1, fig_2], [fig_3, fig_4], [fig_5, fig_6]])

handle_line_chart = show(fig, notebook_handle=True)

In [14]:
while True:
    item = records.get()
    print(item)
    if isinstance(item, Termination):
        print(f"Consumer: Termination signal ({item}) received. Exiting.")
        break
    #print(item)
    elif isinstance(item, TimeStepStart):
        #print(f"Timestep: {item.time_step}, Step size: {item.step_size}")
        new_row = {"step_size": [item.step_size,], "time_step": [item.time_step,], "assembly_time": [0], "linear_solver_time": [0], "step_start_time": [item.step_start_time], "iteration_number": [0]}
        data_source.stream(new_row)
        push_notebook(handle=handle_line_chart)
    elif isinstance(item, AssemblyTime):
        index = len(data_source.data["assembly_time"])-1
        new_time = data_source.data["assembly_time"][index] + item.assembly_time
        data_source.patch({"assembly_time": [(index, new_time)]})
    elif isinstance(item, LinearSolverTime):
        index = len(data_source.data["linear_solver_time"])-1
        new_time = data_source.data["linear_solver_time"][index] + item.linear_solver_time
        data_source.patch({"linear_solver_time": [(index, new_time)]})
    elif isinstance(item, IterationEnd):
        index = len(data_source.data["iteration_number"])-1
        iteration = item.iteration_number
        data_source.patch({"iteration_number": [(index, iteration)]})
    elif isinstance(item, IterationStart):
        #print(data_source_iter.data["iteration_number"][-1], len(data_source_iter.data["iteration_number"]))
        iteration_offset = len(data_source_iter.data["iteration_number"])
        line_width = 0
        if item.iteration_number == 1:
            line_width = 1
        new_row = {"iteration_number": [iteration_offset+1], "vspan": [iteration_offset+0.75], "line_width": [line_width], "dx_x": [1], "dx_x_0": [1], "dx_x_1": [1],"dx_x_2": [1], "dx_x_3": [1], "dx_x_4": [1],"dx_x_5": [1]}
        data_source_iter.stream(new_row)
        push_notebook(handle=handle_line_chart)
    elif isinstance(item, TimeStepConvergenceCriterion):
        index = len(data_source_iter.data["iteration_number"])-1
        data_source_iter.patch({f"dx_x": [(index, item.dx_x)]})
    elif isinstance(item, ComponentConvergenceCriterion):
        index = len(data_source_iter.data["iteration_number"])-1
        data_source_iter.patch({f"dx_x_{item.component}": [(index, item.dx_x)]})
    time.sleep(0.1)
observer.join()

TimeStepStart(type='Info', line=27, mpi_process=0, time_step=0, step_start_time=0.0, step_size=0.0)
TimeStepOutputTime(type='Info', line=28, mpi_process=0, time_step=0, output_time=0.0725941)
TimeStepStart(type='Info', line=31, mpi_process=0, time_step=1, step_start_time=86400.0, step_size=86400.0)
SolvingProcessStart(type='Info', line=32, mpi_process=0, process=0)
IterationStart(type='Info', line=33, mpi_process=0, iteration_number=1)
AssemblyTime(type='Info', line=35, mpi_process=0, assembly_time=0.113481)
DirichletTime(type='Info', line=36, mpi_process=0, dirichlet_time=0.00525803)
LinearSolverTime(type='Info', line=43, mpi_process=0, linear_solver_time=0.0641466)
ComponentConvergenceCriterion(type='Info', line=44, mpi_process=0, component=0, dx=13609000.0, x=7208900000.0, dx_x=0.0018877)
ComponentConvergenceCriterion(type='Info', line=45, mpi_process=0, component=1, dx=0.0059463, x=0.0059463, dx_x=1.0)
ComponentConvergenceCriterion(type='Info', line=46, mpi_process=0, component=2, 

KeyboardInterrupt: 

In [ ]:
from bokeh.io import show
from bokeh.plotting import figure

plot = figure()

plot.hspan(
    y=[0, 5, 15, 33],
    line_width=[1, 2, 3, 4], line_color="red",
)
plot.vspan(
    x=[0, 5, 15, 33],
    line_width=[1, 2, 3, 0], line_color="blue",
)

show(plot)
